In [ ]:
# https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.NaiveBayes.html#


In [35]:
import findspark

findspark.init()

In [36]:
# spark 생성

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [37]:
data = spark.read.csv("hdfs://localhost:19000/data/model_data.csv", header="true", inferSchema="true")

In [38]:
data.show()

+---+-----+-----+-----+-----+-----+------+---------+---------+
|win|topwr| jgwr|midwr|adcwr|supwr|teamwr|recentwr1|recentwr2|
+---+-----+-----+-----+-----+-----+------+---------+---------+
|  0| 50.0| 50.0| 41.2| 50.0| 45.7|  38.5|     66.7|    100.0|
|  0| 44.8| 33.3| 47.0| 45.8| 43.3|  36.7|     57.1|     71.4|
|  0| 28.6| 29.2| 66.7| 22.2| 22.2|  22.2|     50.0|    100.0|
|  0| 35.3| 66.7| 85.7| 50.0| 58.3|  35.3|     50.0|     57.1|
|  0| 55.6| 42.1| 50.0| 40.0| 69.2|  53.3|     85.7|     33.3|
|  1| 92.3| 88.2| 66.7| 92.3| 84.6|  92.3|     37.5|     57.1|
|  0| 85.7| 50.0| 60.0| 66.7| 90.0|  70.1|     75.0|     14.3|
|  1| 47.6| 60.0| 61.9| 50.0| 60.0|  50.0|     71.4|     40.0|
|  1| 75.0|100.0| 92.9|100.0| 78.9|  70.6|    100.0|     54.5|
|  0| 40.0| 40.0| 50.0| 50.0| 50.0|  57.1|     30.0|     50.0|
|  1| 80.0| 83.3| 62.1| 75.0| 71.4|  60.6|     42.9|     25.0|
|  0| 38.9| 40.0| 66.7| 50.0| 50.0|  53.6|     62.5|     28.6|
|  0| 80.0| 50.0| 58.6| 40.0| 40.5|  54.0|     42.9|   

In [39]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

In [40]:
# feature 묶기
assembler = VectorAssembler(
    inputCols=data.columns[1:],
    outputCol="features")

data = assembler.transform(data)

# 데이터 분리
train_data, test_data = data.randomSplit(weights=[0.8, 0.2],  seed=13)

In [66]:
# 모델 생성 & 학습 - multinomial
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol='features',labelCol='win')
model = nb.fit(train_data)

In [88]:
# 샘플 출력

predictions = model.transform(test_data)
predictions.select(['win','prediction','probability']).show(20, False)

+---+----------+------------------------------------------+
|win|prediction|probability                               |
+---+----------+------------------------------------------+
|0  |0.0       |[0.999999641056474,3.589435259030957E-7]  |
|0  |0.0       |[0.9999937853645334,6.214635466579712E-6] |
|0  |1.0       |[0.004944317255356589,0.9950556827446434] |
|0  |0.0       |[0.9999999281653169,7.183468305303433E-8] |
|0  |0.0       |[0.9999823577238759,1.7642276124168257E-5]|
|0  |0.0       |[0.9999986048530285,1.3951469715277848E-6]|
|0  |0.0       |[0.9999986581411819,1.3418588180529717E-6]|
|0  |0.0       |[0.9999971179755779,2.882024422050082E-6] |
|0  |0.0       |[0.9893325980501351,0.010667401949864942] |
|0  |1.0       |[3.7052767072770005E-4,0.9996294723292722]|
|0  |0.0       |[0.9853629030437145,0.014637096956285421] |
|0  |0.0       |[0.7749760028320176,0.22502399716798244]  |
|0  |0.0       |[0.9008460489415753,0.09915395105842481]  |
|0  |1.0       |[1.3582763845137959E-4,0

In [76]:
# 정확도 측정

evaluator = MulticlassClassificationEvaluator(labelCol="win", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6875


In [54]:
# 모델 생성 & 학습 - bernoulli
nb = NaiveBayes(smoothing=1.0, modelType="bernoulli", featuresCol='features',labelCol='win')
model = nb.fit(train_data)


# 정확도 측정
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="win", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Py4JJavaError: An error occurred while calling o1076.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 1445, DESKTOP-CMASS6R, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(NaiveBayes$$Lambda$3138/0x0000000100fe8040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.ScalaUDF.eval(ScalaUDF.scala:1130)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:338)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.update(Summarizer.scala:308)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.update(interfaces.scala:552)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator$$anonfun$1.$anonfun$applyOrEl